# Выбор локации для скважины

В ходе проекта необходимо спрогнозировать место для бурения.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Загрузим имеющиеся у нас в распоряжении 3 датасета, изучим их составляющие, оценим данные с точки зрения логики и рассмотрим как они коррелируют между собой.

In [1]:
#импортируем требуемые библиотеки
import pandas as pd
import numpy as np
from random import randrange
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score

In [2]:
#загрузим датасет
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [4]:
data_0.drop(['id'], axis=1).corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [5]:
#загрузим датасет
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [7]:
data_1.drop(['id'], axis=1).corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [8]:
#загрузим датасет
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
data_2.drop(['id'], axis=1).corr().head()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


In [9]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [10]:
data_2.drop(['id'], axis=1).corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Как мы видим данные представленные в трёх датасетах соответствующие регионам "0", "1" и "2" имеют корреляцию с параметром f2, особенно в регионе "1" где зависимость практически прямопропорциональна (смеем сделать вывод, что ML-модель обучаемая нами в дальнейшем покажет наилучший результат с точки зрения точности и минимальному среднеквадратическому отклонению именно по региону "1").

## Обучение и проверка модели

Разобьём датасеты на обучающую и валидационную части, а также обучим методом линейной регрессии и сравним обученные модели из разных регионов на основании среднеквадратической ошибки предсказания.

In [11]:
target_0 = data_0['product']
features_0 = data_0.drop(['product','id'] , axis=1)
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

In [12]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_valid_0 = model_0.predict(features_valid_0)
mse_0 = mean_squared_error(target_valid_0, predictions_valid_0) 

print('MAE: ', mean_absolute_error(target_valid_0, predictions_valid_0)) 
print("RMSE: {}".format(np.sqrt(mse_0)))
print('Средний заказ предсказанного сырья по региону "0":', predictions_valid_0.mean())

MAE:  30.919600777151313
RMSE: 37.5794217150813
Средний заказ предсказанного сырья по региону "0": 92.59256778438035


In [13]:
target_1 = data_1['product']
features_1 = data_1.drop(['product','id'] , axis=1)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

In [14]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_valid_1 = model_1.predict(features_valid_1)
mse_1 = mean_squared_error(target_valid_1, predictions_valid_1) 
print('MAE: ', mean_absolute_error(target_valid_1, predictions_valid_1)) 
print("RMSE: {}".format(np.sqrt(mse_1)))
print('Средний заказ предсказанного сырья по региону "1":', predictions_valid_1.mean())

MAE:  0.7187662442124758
RMSE: 0.893099286775617
Средний заказ предсказанного сырья по региону "1": 68.728546895446


In [15]:
target_2 = data_2['product']
features_2 = data_2.drop(['product','id'] , axis=1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

In [16]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_valid_2 = model_2.predict(features_valid_2)
mse_2 = mean_squared_error(target_valid_2, predictions_valid_2) 
print('MAE: ', mean_absolute_error(target_valid_2, predictions_valid_2)) 
print('RMSE: {}'.format(np.sqrt(mse_2)))
print('Средний заказ предсказанного сырья по региону "2":', predictions_valid_2.mean())

MAE:  32.792652105481814
RMSE: 40.02970873393434
Средний заказ предсказанного сырья по региону "2": 94.96504596800489


Как мы и предполагали наименьшая среднеквадратическая ошибка показана на линейной модели в регионе №1 ввиду прямопропорциональной зависимости данных.

## Подготовка к расчёту прибыли

Рассмотрим основные условия в постановке задачи:

In [17]:
#условия задачи:
#При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
#После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. 
#Среди них выбирают регион с наибольшей средней прибылью.

CATCH_POINTS = 500
BUDGET = 10000000000 #бюджет разработки скважин в регионе
PRICE = 450000 #цена рублей за тысячу баррелей нефти 

Рассчитаем точку безубыточности объёма средних залежей нефти по лучшим 200 скважинам предсказанным моделью на месторождении для покрытия затрат на разведывательные работы. 

In [18]:
breakeven = BUDGET/PRICE/200
print(breakeven, 'тыс.баррелей нефти')

111.11111111111111 тыс.баррелей нефти


Соответственно, необходимо определить регион где топ-200 скважин имеют средние запасы нефти в каждой точке более 111 тысяч барелей.

Сформируем топ-200 скважин по всем трём регионам для оценки средних максимальных запасов которые есть по факту и предсказанные:

In [19]:
predictions_0 = model_0.predict(features_0)
target_0_top = target_0.sort_values(ascending=False).reset_index(drop=True).loc[:199]
predictions_0_top = pd.Series(predictions_0).sort_values(ascending=False).reset_index(drop=True).loc[:199]
print('Средние топ-200 объёмы скважин региона №0 (истинные):',target_0_top.mean())
print('Средние топ-200 объёмы скважин региона №0 (предсказанные):',predictions_0_top.mean())

Средние топ-200 объёмы скважин региона №0 (истинные): 184.83373964536008
Средние топ-200 объёмы скважин региона №0 (предсказанные): 163.2177352995591


In [20]:
predictions_1 = model_1.predict(features_1)
target_1_top = target_1.sort_values(ascending=False).reset_index(drop=True).loc[:199]
predictions_1_top = pd.Series(predictions_1).sort_values(ascending=False).reset_index(drop=True).loc[:199]
print('Средние топ-200 объёмы скважин региона №1 (истинные):',target_1_top.mean())
print('Средние топ-200 объёмы скважин региона №1 (предсказанные):',predictions_1_top.mean())

Средние топ-200 объёмы скважин региона №1 (истинные): 137.9454077409057
Средние топ-200 объёмы скважин региона №1 (предсказанные): 139.16229068572267


In [21]:
predictions_2 = model_0.predict(features_2)
target_2_top = target_2.sort_values(ascending=False).reset_index(drop=True).loc[:199]
predictions_2_top = pd.Series(predictions_2).sort_values(ascending=False).reset_index(drop=True).loc[:199]
print('Средние топ-200 объёмы скважин региона №2 (истинные):',target_2_top.mean())
print('Средние топ-200 объёмы скважин региона №2 (предсказанные):',predictions_2_top.mean())

Средние топ-200 объёмы скважин региона №2 (истинные): 189.55147698176646
Средние топ-200 объёмы скважин региона №2 (предсказанные): 202.9868109503039


## Расчёт прибыли и рисков 

Напишем функцию, которая расчитает выручку по заданному количеству месторождений на основании прогнозных и истинных данных:

In [22]:
def profit(target, predictions, count):
    probs_sorted = pd.Series(predictions).sort_values(ascending=False)
    result = target[probs_sorted.index][:count].sum()
    return 450000 * result - BUDGET

Рассчитаем для понимания порядков значений выручку по топ 200 точек каждого региона: 

In [23]:
print('Выручка по региону №0 составит:',profit(target_0,predictions_0,200)/1000000000, 'млрд.руб.')

Выручка по региону №0 составит: 3.494104192112015 млрд.руб.


In [24]:
print('Выручка по региону №1 составит:',profit(target_1,predictions_1,200)/1000000000,'млрд.руб.')

Выручка по региону №1 составит: 2.4150866966815108 млрд.руб.


In [25]:
print('Выручка по региону №2 составит:',profit(target_2,predictions_2,200)/1000000000,'млрд.руб.')

Выручка по региону №2 составит: 3.2460290047726974 млрд.руб.


Оченим техников Bootstrap распределение на 1000 выборок для каждого региона для выбора наиболее прогнозируемого и менее рискованного объекта инвестиций:

In [30]:
state = np.random.RandomState(12345)
target_valid_0 = target_valid_0.reset_index(drop=True)
predictions_valid_0 = pd.Series(predictions_valid_0)
target_valid_1 = target_valid_1.reset_index(drop=True)
predictions_valid_1 = pd.Series(predictions_valid_1)
target_valid_2 = target_valid_2.reset_index(drop=True)
predictions_valid_2 = pd.Series(predictions_valid_2)

In [31]:

values_0 = []

for i in range(1000):
    
    target_subsample_0 = target_valid_0.sample(n=500, replace=True, random_state=state)
    pred_subsample_0 = predictions_valid_0[target_subsample_0.index]
    res_0 = profit(target_subsample_0,pred_subsample_0,200)  
    values_0.append(res_0)

values_0 = pd.Series(values_0)
upper_0 = values_0.quantile(0.975)
lower_0 = values_0.quantile(q=0.025)
mean_0 = values_0.mean()
print("Средняя прибыль/убыток:", mean_0/1000000000,'млрд.руб.')
print("2,5%-квантиль:", lower_0/1000000000,'млрд.руб.')
print("97,5%-квантиль:", upper_0/1000000000,'млрд.руб.')
print("Диапазон изменения прибыли составил:", (upper_0-lower_0)/1000000000,'млрд.руб.')


schet_0=0
for i in range(len(values_0)):
    if values_0[i]<0:
        schet_0+=1
print('Риск получения убытка составляет:',schet_0/1000*100, '%')

Средняя прибыль/убыток: 0.4259385269105924 млрд.руб.
2,5%-квантиль: -0.10209009483793653 млрд.руб.
97,5%-квантиль: 0.9479763533583688 млрд.руб.
Диапазон изменения прибыли составил: 1.0500664481963053 млрд.руб.
Риск получения убытка составляет: 6.0 %


In [33]:
values_1 = []

for i in range(1000):
    target_subsample_1 = target_valid_1.sample(n=500, replace=True, random_state=state)
    pred_subsample_1 = predictions_valid_1[target_subsample_1.index] 
    res_1 = profit(target_subsample_1,pred_subsample_1,200)  
    values_1.append(res_1)

values_1 = pd.Series(values_1)
upper_1 = values_1.quantile(0.975)
lower_1 = values_1.quantile(q=0.025)
mean_1 = values_1.mean()
print("Средняя прибыль/убыток:", mean_1/1000000000,'млрд.руб.')
print("2,5%-квантиль:", lower_1/1000000000,'млрд.руб.')
print("97,5%-квантиль:", upper_1/1000000000,'млрд.руб.')
print("Диапазон изменения прибыли составил:", (upper_1-lower_1)/1000000000,'млрд.руб.')

schet_1 = 0
for i in range(len(values_0)):
    if values_1[i] < 0:
        schet_1 += 1
print('Риск получения убытка составляет:',schet_1/1000*100, '%')

Средняя прибыль/убыток: 0.5068142889989468 млрд.руб.
2,5%-квантиль: 0.11023611441505556 млрд.руб.
97,5%-квантиль: 0.940089183344335 млрд.руб.
Диапазон изменения прибыли составил: 0.8298530689292795 млрд.руб.
Риск получения убытка составляет: 0.8 %


In [34]:
values_2 = []

for i in range(1000):
    target_subsample_2 = target_valid_2.sample(n=500, replace=True, random_state=state)
    pred_subsample_2= predictions_valid_2[target_subsample_2.index] 
    res_2 = profit(target_subsample_2,pred_subsample_2,200)  
    values_2.append(res_2)

values_2 = pd.Series(values_2)


upper_2 = values_2.quantile(0.975)
lower_2 = values_2.quantile(q=0.025)
mean_2 = values_2.mean()
print("Средняя прибыль/убыток:", mean_2/1000000000,'млрд.руб.')
print("2,5%-квантиль:", lower_2/1000000000,'млрд.руб.')
print("97,5%-квантиль:", upper_2/1000000000,'млрд.руб.')
print("Диапазон изменения прибыли составил:", (upper_2-lower_2)/1000000000,'млрд.руб.')

schet_2 = 0
for i in range(len(values_0)):
    if values_2[i] < 0:
        schet_2 += 1
print('Риск получения убытка составляет:',schet_2/1000*100, '%')

Средняя прибыль/убыток: 0.42421248809915973 млрд.руб.
2,5%-квантиль: -0.0949202684330111 млрд.руб.
97,5%-квантиль: 0.9328900216850643 млрд.руб.
Диапазон изменения прибыли составил: 1.0278102901180755 млрд.руб.
Риск получения убытка составляет: 4.9 %


## Основные выводы по проекту

Как мы видим, после применения технологии Bootstrap был получен результат: регион №1 оказался регионом с наибольшей средней прибылью и с наименьшими рисками (риск получения убытка составил лишь 0,8%, а в сфере бизнеса такой риск можно считать приемлимым). Помимо всего прочего с самого начала проекта мы отмечали, что объём залежей региона №1 наиболее предсказуем ввиду практически прямопропорциональной корреляции одного из показателей.

В ходе проекта мы отпработали технологию для оценки прибыльности месторождения и данную технологию можно тиражировать на разные  регионы для проведения геологоразведывательных работ.